In [1]:
import numpy as np
import pandas as pd
import pymysql
import time

In [2]:
connection = pymysql.connect("localhost", "root", "", "moviereviews")

movies = pd.read_sql_query("SELECT * from movies_movies", connection)
ratings = pd.read_sql_query("SELECT * from movies_ratings", connection)

In [3]:
movies.head()

,id,title,genres
0,1,Toy Story 2,Comedy | Action
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,6,Heat (1995),Action|Crime|Thriller


In [4]:
ratings.head()

,id,rating,movieId_id,userId_id,timestamp
0,1,4.5,1,1,0
1,2,4.0,1,1,964982703
2,3,4.0,3,1,964981247
3,4,4.0,6,1,964982224
4,5,5.0,47,1,964983815


In [9]:
def create_ranked_df(movies, reviews):
    movie_ratings = ratings.groupby('movieId_id')['rating']
    avg_ratings = movie_ratings.mean()
    num_ratings = movie_ratings.count()
    last_rating = ratings.groupby('movieId_id').max()['timestamp']
    last_rating.columns = ['last_rating']
    rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
    rating_count_df = rating_count_df.join(last_rating)
    movie_recs = movies.set_index('id').join(rating_count_df)
    ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'timestamp'], ascending=False)
    ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
    
    return ranked_movies

In [10]:
ranked_movies = create_ranked_df(movies, ratings)
ranked_movies.head()

,title,genres,avg_rating,num_ratings,timestamp
id,,,,,
1222,Stand by Me (1986),Adventure|Drama,4.7000,5.0,1.455050e+09
1270,Hype! (1996),Documentary,4.6875,8.0,1.510572e+09
3147,Bless the Child (2000),Thriller,4.6000,5.0,1.510572e+09
590,Dances with Wolves (1990),Adventure|Drama|Western,4.5000,7.0,1.455210e+09
923,Citizen Kane (1941),Drama|Mystery,4.5000,5.0,1.455060e+09
